In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/응용통계학과 공모전')

# 요약
1. 저장된 SBERT 모델, K-means++ 모델을 로드
2. 사용자 Input을 문장별로 Text Embedding후 라벨 할당
3. 생성형AI, Text NetWork, 채용공고 추천 등 활용

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import pickle
from tqdm import tqdm
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel

In [ ]:
# 모델 로드
loaded_AutoModel = AutoModel.from_pretrained('./model/AutoModel_final')
loaded_AutoTokenizer = AutoTokenizer.from_pretrained('./model/AutoTokenizer_final')
loaded_kmeans = torch.load('./model/kmeans_final')

In [ ]:
# Input_prediction함수
def predict_clusters(new_data, loaded_AutoModel, loaded_AutoTokenizer, loaded_kmeans):
    # Input 문장을 토큰화하고 BERT 임베딩
    tokenized_new_data = [loaded_AutoTokenizer(sentence, return_tensors='pt', padding=True, truncation=True) for sentence in new_data]
    encoded_new_data = [loaded_AutoModel(**tokens).last_hidden_state.mean(dim=1).detach().numpy() for tokens in tokenized_new_data]

    # float32로 형변환
    encoded_new_data = [embedding.astype('float32') for embedding in encoded_new_data]
    loaded_kmeans.cluster_centers_ = loaded_kmeans.cluster_centers_.astype('float32')

    # 각 새로운 문장에 대해 클러스터를 예측
    new_data_clusters = loaded_kmeans.predict(np.vstack(encoded_new_data))

    return new_data_clusters

In [ ]:
# Input sentence example
new_data = ["Pytorch를 능숙하게 사용가능", '파이프라인 구축 경험이 있음', 'Python을 능숙하게 사용함']

# Input prediction
new_data_clusters = predict_clusters(new_data, loaded_AutoModel, loaded_AutoTokenizer, loaded_kmeans)

for sentence, cluster in zip(new_data, new_data_clusters):
    print(f"Sentence: {sentence}, Predicted Cluster: {cluster}")

Sentence: Pytorch를 능숙하게 사용가능, Predicted Cluster: 2
Sentence: 파이프라인 구축 경험이 있음, Predicted Cluster: 3
Sentence: Python을 능숙하게 사용함, Predicted Cluster: 2
